Let $S(k)$ be the sum of three or more distinct positive integers having the following properties:


No value exceeds k.
The values form a geometric progression.
The sum is maximal.

$S(4) = 4 + 2 + 1 = 7$

$S(10) = 9 + 6 + 4 = 19$

$S(12) = 12 + 6 + 3 = 21$

$S(1000) = 1000 + 900 + 810 + 729 = 3439$

Let $T(n)= \sum_{k=4}^{n}(−1)^{k}S(k)$.

$T(1000) = 2268$

Find $T(10^{17})$.

### Facts

$S(k) = n(1 + r + r^2)$

For a geometric series $(nr^2, nr, n) = (a, b, c)$ we have that $b^2 = ac$

### Ideas
##### 3 case 
Find the prime decomposition of $k = p_1^{\gamma_1} \cdot ... \cdot p_l^{\gamma_l}$ and find the smallest square we can take out, let $k'$ be $k$ divided by this smallest factor. This insures that $k \cdot k'$ is still square. This is our first best guess at $c$ (since we know $ac$ is definately square by this procedure). Then find the factor $f = \frac{k}{k'}$, we know that $fk' \leq k$. so we can find the largest square number $\hat{f} \leq f$ and we have that (obviously) $\hat{f}k' \leq k'$ and since $\hat{f}$ is square $\hat{f}k'k$ is also still square and $b = \sqrt{\hat{f}k'k}$

In [144]:
import numpy as np
import primefac as pf
from __future__ import division
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime


In [212]:
# Helper function to get prime decomposition
def make_small(factors):
    # we are going to find the smallest power of 2 and subtract it
    for key in factors:
        if factors[key] > 1:
            factors[key] = factors[key] - 2
            return factors
    return factors

def make_even(factors):
    for key in factors:
        return np.array([count - (count % 2) for count in counts])

def multiply(factors):
    prod = 1
    for key in factors:
        prod *= key ** factors[key]
    return int(prod)

time: 5.7 ms


In [213]:
def s(k):
    # Base case
    if k == 4:
        return [4, 2, 1]
    
    # Lower bound from s(k-1)
    lower_bound = np.sum(s(k-1))
    
    # Find our first guess of n (the smallest of the series) then improve
    prime_factors = pf.factorint(k)
    n_factors = make_small_(prime_factors)
    n = multiply(n_factors)
    factor = k // n
    if factor > 1:
        square_factor = pf.introot(n, r=2)
        n *= square_factor
        return [k, int(np.sqrt(n * k)), n]
    else:
        return s(k-1)

time: 6.44 ms


In [336]:
def get_highest_two(factors):
    highest = {}
    for key in factors:
        if factors[key] >= 2:
            highest[key] = factors[key]
    for key in highest:
        highest[key] = (highest[key] - (highest[key] % 2)) / 2
    return highest

def to_seq(k, numerator):
    denominator = numerator - 1
    seq = [k]
    elem = k
    while (elem * denominator) % numerator == 0:
        elem = (elem * denominator) / numerator
        seq.append(int(elem))
    return seq

def get_r(k):
    factors = pf.factorint(k)
    numerator = multiply(get_highest_two(factors))
    return numerator, numerator - 1

def s_(k):
    if k == 4:
        return [4, 2, 1]
    numerator, denominator = get_r(k)
    l_numerator, l_denomintor = get_r(k - 1)
    if denominator > 0: # we have a sequence where k is at least ar^{3}
        if l_denomintor > 0:
            if (l_numerator/(l_numerator - 1)) <= (numerator/(numerator - 1)):
                base_seq = s(k - 1)
                seq = to_seq(k, numerator)
                if np.sum(base_seq) > np.sum(seq):
                    return base_seq
                else:
                    return seq
        return to_seq(k, numerator)
    else:
        return s(k-1)

def s(k):
    if k == 4:
        return [4, 2, 1]
    numerator, denominator = get_r(k)
    if denominator > 0: # we have a sequence where k is at least ar^{3}
        if len(pf.factorint(numerator)) < 3:
            base_seq = s(k - 1)
            seq = to_seq(k, numerator)
            if np.sum(base_seq) > np.sum(seq):
                return base_seq
            else:
                return seq
        return to_seq(k, numerator)
    else:
        return s(k-1)
    
def t(n):
    total = 0
    for i in range(4, n + 1):
        total += ((-1)**i) * np.sum(s(i))
    return total

time: 42.7 ms


In [337]:
s(100)

[100, 90, 81]

time: 3.8 ms


In [338]:
l_seq = [0]
for i in range(4,1000):
    seq = s(i)
    if np.sum(seq) < np.sum(l_seq):
        print i-1, l_seq, np.sum(l_seq)
        print '*', i, seq, np.sum(seq)
    l_seq = seq

time: 6.64 s


In [335]:
pf.factorint(968)

{2: 3, 11: 2}

time: 1.74 ms


In [102]:
np.sum(s(23456))

4395